# 📈 Forecast Fill Level with Prophet
This notebook reads sensor data from S3, trains a time-series model using Facebook Prophet, and uploads 24-hour forecasts back to S3.

In [ ]:

# 📦 Install required packages (if needed)
!pip install --quiet pandas prophet boto3 s3fs


In [ ]:

import pandas as pd
from prophet import Prophet
import boto3

# S3 input path
s3_input_path = 's3://co2-plant-processed-data/forecasting/input/forecast_data.csv'

# Load CSV directly from S3
df = pd.read_csv(s3_input_path)
df = df[['timestamp', 'fill_level']].copy()
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values('ds')
df.head()


In [ ]:

# 🔮 Train Prophet model and forecast next 24 hours
model = Prophet()
model.fit(df)

future = model.make_future_dataframe(periods=24, freq='H')
forecast = model.predict(future)

forecast_result = forecast[['ds', 'yhat']].copy()
forecast_result.head()


In [ ]:

# 💾 Save forecast locally
output_file = '/tmp/fill_level_forecast.csv'
forecast_result.to_csv(output_file, index=False)


In [ ]:

# ☁️ Upload to S3 output folder
s3 = boto3.client('s3')
s3.upload_file(
    Filename=output_file,
    Bucket='co2-plant-processed-data',
    Key='forecasting/output/fill_level_forecast.csv'
)

print("✅ Forecast uploaded to s3://co2-plant-processed-data/forecasting/output/fill_level_forecast.csv")
